<a href="https://colab.research.google.com/github/mantoan-thi/Day_trade_mini_indice/blob/main/Day_trade_mini_indice.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import MetaTrader5 as mt5
import pandas as pd
import numpy as np
from talib.abstract import *
import matplotlib.pyplot as plt
import datetime, time
%matplotlib inline
import plotly.graph_objects as go
import seaborn as sns
import random
from datetime import timedelta, date
import time
from datetime import datetime
import plotly.express as px
import seaborn as sns
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score, recall_score, confusion_matrix,classification_report
from sklearn.preprocessing import StandardScaler

In [ ]:
if not mt5.initialize(login=3002637411, server="Rico-Demo",password="heidegger*"):
    print("initialize() failed, error code =",mt5.last_error())
    quit()

In [ ]:
def obter_precos_treino(par,time_,modo):
    if time_ == 'M1':timeframe = mt5.TIMEFRAME_M1
    if time_ == 'M5':timeframe = mt5.TIMEFRAME_M5
    if time_ == 'M15':timeframe = mt5.TIMEFRAME_M15
    if time_ == 'H1':timeframe = mt5.TIMEFRAME_H1
    if time_ == 'H2':timeframe = mt5.TIMEFRAME_H2
    if time_ == 'H3':timeframe = mt5.TIMEFRAME_H3
    if time_ == 'H4':timeframe = mt5.TIMEFRAME_H4
    if time_ == 'H6':timeframe = mt5.TIMEFRAME_H6
    if time_ == 'H8':timeframe = mt5.TIMEFRAME_H8
    if time_ == 'H12':timeframe = mt5.TIMEFRAME_H12
    if time_ == 'D1':timeframe = mt5.TIMEFRAME_D1
    if time_ == 'W1': timeframe = mt5.TIMEFRAME_W1
    if time_ == 'MN1':timeframe = mt5.TIMEFRAME_MN1
    rates = pd.DataFrame(mt5.copy_rates_from_pos(par,timeframe,200,5000))
    rates.drop(columns=['spread','real_volume'], inplace=True)
    rates['time'] = pd.to_datetime(rates['time'],unit='s')
    if modo == 'Compra':
        rates['Retorno'] = (rates['close'].shift(-4)/rates['close'].shift(-1)-1)*10000
        _,_, rates['lowerband'] = BBANDS(rates.close, timeperiod=20, nbdevup=2.0, nbdevdn=2.0, matype=0)
        rates['RSI'] = RSI(rates.close, timeperiod=14)
        rates['macd'], rates['macdsignal'],_ = MACD(rates.close, fastperiod=12, slowperiod=26, signalperiod=9)
        rates['slowk'], rates['slowd'] = STOCH(rates.high, rates.low, rates.close, fastk_period=5, slowk_period=3, slowk_matype=0, slowd_period=3, slowd_matype=0)
        #rates['MOM'] = MOM(rates.close, timeperiod=10)*10000
        #rates['Media_9'] = SMA(rates.close, timeperiod=9)
        #rates['Media_21'] = SMA(rates.close, timeperiod=21)
    if modo == 'Venda':
        rates['Retorno'] = (rates['close'].shift(-4)/rates['close'].shift(-1)-1)*10000  
        rates['upperband'],_,_ = BBANDS(rates.close, timeperiod=20, nbdevup=2.5, nbdevdn=2.5, matype=0)
        rates['RSI'] = RSI(rates.close, timeperiod=14)
        rates['macd'], rates['macdsignal'],_ = MACD(rates.close, fastperiod=12, slowperiod=26, signalperiod=9)
        rates['slowk'], rates['slowd'] = STOCH(rates.high, rates.low, rates.close, fastk_period=5, slowk_period=3, slowk_matype=0, slowd_period=3, slowd_matype=0)
        #rates['MOM'] = MOM(rates.close, timeperiod=10)*10000
        #rates['Media_9'] = SMA(rates.close, timeperiod=9)
        #rates['Media_21'] = SMA(rates.close, timeperiod=21)
    rates = rates[(rates['Retorno']>30)]
    rates.dropna(inplace=True)
    return rates

In [ ]:
def obter_precos_test(par,time_,modo):
    if time_ == 'M1':timeframe = mt5.TIMEFRAME_M1
    if time_ == 'M5':timeframe = mt5.TIMEFRAME_M5
    if time_ == 'M15':timeframe = mt5.TIMEFRAME_M15
    if time_ == 'H1':timeframe = mt5.TIMEFRAME_H1
    if time_ == 'H2':timeframe = mt5.TIMEFRAME_H2
    if time_ == 'H3':timeframe = mt5.TIMEFRAME_H3
    if time_ == 'H4':timeframe = mt5.TIMEFRAME_H4
    if time_ == 'H6':timeframe = mt5.TIMEFRAME_H6
    if time_ == 'H8':timeframe = mt5.TIMEFRAME_H8
    if time_ == 'H12':timeframe = mt5.TIMEFRAME_H12
    if time_ == 'D1':timeframe = mt5.TIMEFRAME_D1
    if time_ == 'W1': timeframe = mt5.TIMEFRAME_W1
    if time_ == 'MN1':timeframe = mt5.TIMEFRAME_MN1
    rates = pd.DataFrame(mt5.copy_rates_from_pos(par,timeframe,0,199))
    rates.drop(columns=['spread','real_volume'], inplace=True)
    rates['time'] = pd.to_datetime(rates['time'],unit='s')
    if modo == 'Compra':
        rates['Retorno'] = (rates['close'].shift(-4)/rates['close'].shift(-1)-1)*10000
        _,_, rates['lowerband'] = BBANDS(rates.close, timeperiod=20, nbdevup=2.0, nbdevdn=2.0, matype=0)
        rates['RSI'] = RSI(rates.close, timeperiod=14)
        rates['macd'], rates['macdsignal'],_ = MACD(rates.close, fastperiod=12, slowperiod=26, signalperiod=9)
        rates['slowk'], rates['slowd'] = STOCH(rates.high, rates.low, rates.close, fastk_period=5, slowk_period=3, slowk_matype=0, slowd_period=3, slowd_matype=0)
        #rates['MOM'] = MOM(rates.close, timeperiod=10)*10000
        #rates['Media_9'] = SMA(rates.close, timeperiod=9)
        #rates['Media_21'] = SMA(rates.close, timeperiod=21)
        #rates = rates[(rates['Retorno']>0)]
        #rates.dropna(inplace=True)
    if modo == 'Venda':
        rates['Retorno'] = (rates['close'].shift(-4)/rates['close'].shift(-1)-1)*10000
        rates['upperband'],_,_ = BBANDS(rates.close, timeperiod=20, nbdevup=2.0, nbdevdn=2.0, matype=0)
        rates['RSI'] = RSI(rates.close, timeperiod=14)
        rates['macd'], rates['macdsignal'],_ = MACD(rates.close, fastperiod=12, slowperiod=26, signalperiod=9)
        rates['slowk'], rates['slowd'] = STOCH(rates.high, rates.low, rates.close, fastk_period=5, slowk_period=3, slowk_matype=0, slowd_period=3, slowd_matype=0)
        #rates['MOM'] = MOM(rates.close, timeperiod=10)*10000
        #rates['Media_9'] = SMA(rates.close, timeperiod=9)
        #rates['Media_21'] = SMA(rates.close, timeperiod=21)
        #rates = rates[(rates['Retorno']>0)]
        #rates.dropna(inplace=True)
    return rates

In [ ]:
def format(dados,modo):
    if modo=='Compra':
        dados['macd2']=dados['macd'].shift(1)
        dados['macdsignal2']=dados['macdsignal'].shift(1)
        dados['slowk2']=dados['slowk'].shift(1)
        dados['slowd2']=dados['slowd'].shift(1)
        #dados['MOM2']=dados['MOM'].shift(1)
        #dados['Media_92']=dados['Media_9'].shift(1)
        #dados['Media_212']=dados['Media_21'].shift(1)
        dados['Sinal_Bandas'] = dados.apply(lambda x: 1 if x['close'] < x['lowerband'] and x['Retorno'] > 0 else 0,axis=1)
        dados['Sinal_RSI'] = dados.apply(lambda x: 1 if x['RSI'] < 30 and x['Retorno'] > 0 else 0,axis=1)
        dados['Sinal_MACD'] = dados.apply(lambda x: 1 if x['macd'] > x['macdsignal'] and x['macd2'] < x['macdsignal2'] and x['Retorno'] > 0 else 0,axis=1)
        dados['Sinal_STOCH'] = dados.apply(lambda x: 1 if x['slowk'] < 40 and x['slowd'] < 40 and x['slowk'] > x['slowd'] and x['slowk2'] < x['slowd2'] and x['Retorno'] > 0 else 0,axis=1)
        #dados['Sinal_MOM'] = dados.apply(lambda x: 1 if x['MOM'] > -20 and x['MOM2'] < -20 and x['Retorno'] > 0 else 0,axis=1)
        #dados['Sinal_MEDIA'] = dados.apply(lambda x: 1 if x['Media_9'] > x['Media_21'] and x['Media_92'] < x['Media_212'] and x['Retorno'] > 0 else 0,axis=1)
        dados['Total'] = dados['Sinal_Bandas']+dados['Sinal_RSI']+dados['Sinal_MACD']+dados['Sinal_STOCH']#+dados['Sinal_MOM']+dados['Sinal_MEDIA']
        dados['Total']=dados.apply(lambda x: 1 if x['Total']!= 0 else 0,axis=1)
        dados.dropna(inplace=True)
        qtde_ex = (dados.shape[0]-(dados['Total'].sum()*2))
        dados.drop(dados[dados['Total']==0][:qtde_ex].index,inplace=True)
    if modo=='Venda':
        dados['macd2']=dados['macd'].shift(1)
        dados['macdsignal2']=dados['macdsignal'].shift(1)
        dados['slowk2']=dados['slowk'].shift(1)
        dados['slowd2']=dados['slowd'].shift(1)
        #dados['MOM2']=dados['MOM'].shift(1)
        #dados['Media_92']=dados['Media_9'].shift(1)
        #dados['Media_212']=dados['Media_21'].shift(1)
        dados['Sinal_Bandas'] = dados.apply(lambda x: 1 if x['close'] > x['upperband'] and x['Retorno'] < 0 else 0,axis=1)
        dados['Sinal_RSI'] = dados.apply(lambda x: 1 if x['RSI'] > 70 and x['Retorno'] > 0 else 0,axis=1)
        dados['Sinal_MACD'] = dados.apply(lambda x: 1 if x['macd'] < x['macdsignal'] and x['macd2'] > x['macdsignal2'] and x['Retorno'] < 0 else 0,axis=1)
        dados['Sinal_STOCH'] = dados.apply(lambda x: 1 if x['slowk'] > 60 and x['slowd'] > 60 and x['slowk'] < x['slowd'] and x['slowk2'] > x['slowd2'] and x['Retorno'] < 0 else 0,axis=1)
        #dados['Sinal_MOM'] = dados.apply(lambda x: 1 if x['MOM'] < 120 and x['MOM2'] > 120 and x['Retorno'] > 0 else 0,axis=1)
        #dados['Sinal_MEDIA'] = dados.apply(lambda x: 1 if x['Media_9'] < x['Media_21'] and x['Media_92'] > x['Media_212'] and x['Retorno'] > 0 else 0,axis=1)
        dados['Total'] = dados['Sinal_Bandas']+dados['Sinal_RSI']+dados['Sinal_MACD']+dados['Sinal_STOCH']#+dados['Sinal_MOM']+dados['Sinal_MEDIA']
        dados['Total']=dados.apply(lambda x: 1 if x['Total']!= 0 else 0,axis=1)
        dados.dropna(inplace=True)
        qtde_ex = (dados.shape[0]-(dados['Total'].sum()*2))
        dados.drop(dados[dados['Total']==0][:qtde_ex].index,inplace=True)
    return dados

# Machine learning Compra

In [ ]:
par = 'WINQ21'

dataset_cp= obter_precos_treino(par,'M5','Compra')
dados = format(dataset_cp,'Compra')
X = dados[['lowerband', 'RSI', 'macd', 'macdsignal', 'slowk', 
                     'slowd', 'macd2', 'macdsignal2', 'slowk2',
                     'slowd2']]
scaler = StandardScaler()

X = scaler.fit_transform(X)
X = scaler.transform(X)

y=dados['Total']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33)

param_grid = {
        "max_depth": [4, 5, 6],
        "max_features": ["auto", "sqrt"],
        "n_estimators": [20,50,100]
    }
cf = RandomForestClassifier()
cf_cv = GridSearchCV(estimator=cf, param_grid=param_grid, cv=5)
cf_cv.fit(X_train, y_train)
param_cp = cf_cv.best_params_

rfc1=RandomForestClassifier(random_state=42,max_features=param_cp['max_features'], n_estimators= param_cp['n_estimators'], max_depth=param_cp['max_depth'])
rfc1.fit(X_train, y_train)
y_pred=rfc1.predict(X_test)
print(accuracy_score(y_test,y_pred))
print(recall_score(y_test,y_pred))
print(confusion_matrix(y_test,y_pred))
print(classification_report(y_test,y_pred))

In [ ]:
par = 'WINQ21'
dataset_vd = obter_precos_treino(par,'M5','Venda')
dados2 = format(dataset_vd,'Venda')
X2 = dados2[['upperband', 'RSI', 'macd', 'macdsignal', 'slowk', 
                 'slowd', 'macd2', 'macdsignal2', 'slowk2',
                 'slowd2']]
y2=dados2['Total']
scaler = StandardScaler()

X2 = scaler.fit_transform(X2)
X2 = scaler.transform(X2)
X_train2, X_test2, y_train2, y_test2 = train_test_split(X2, y2, test_size=0.33)

param_grid = {
    "max_depth": [4, 5, 6],
    "max_features": ["auto", "sqrt"],
    "n_estimators": [20,50,100]
}
cf2 = RandomForestClassifier()
cf_cv2 = GridSearchCV(estimator=cf2, param_grid=param_grid, cv=5)
cf_cv2.fit(X_train2, y_train2)

param_vd = cf_cv2.best_params_

rfc2=RandomForestClassifier(random_state=42,max_features=param_vd['max_features'], n_estimators= param_vd['n_estimators'], max_depth=param_vd['max_depth'])
rfc2.fit(X_train2, y_train2)
y_pred2=rfc2.predict(X_test2)
print(accuracy_score(y_test2,y_pred2))
print(recall_score(y_test2,y_pred2))
print(confusion_matrix(y_test2,y_pred2))
print(classification_report(y_test2,y_pred2))

# Backtest

In [ ]:
def backtest(modo): 
    if modo=='Compra':
        par = 'WINQ21'
        New_dados = obter_precos_test(par,'M5','Compra')
        New_dados['macd2']=New_dados['macd'].shift(1)
        New_dados['macdsignal2']=New_dados['macdsignal'].shift(1)
        New_dados['slowk2']=New_dados['slowk'].shift(1)
        New_dados['slowd2']=New_dados['slowd'].shift(1)
        #New_dados['MOM2']=New_dados['MOM'].shift(1)
        #New_dados['Media_92']=New_dados['Media_9'].shift(1)
        #New_dados['Media_212']=New_dados['Media_21'].shift(1)
        New_dados.dropna(inplace=True)
        New_dados2 = New_dados[['lowerband', 'RSI', 'macd', 'macdsignal', 'slowk', 
                         'slowd', 'macd2', 'macdsignal2', 'slowk2',
                         'slowd2']]
        #New_dados2.dropna(inplace=True)
    if modo=='Venda':
        par = 'WINQ21'
        New_dados = obter_precos_test(par,'M5','Venda')
        New_dados['macd2']=New_dados['macd'].shift(1)
        New_dados['macdsignal2']=New_dados['macdsignal'].shift(1)
        New_dados['slowk2']=New_dados['slowk'].shift(1)
        New_dados['slowd2']=New_dados['slowd'].shift(1)
        #New_dados['MOM2']=New_dados['MOM'].shift(1)
        #New_dados['Media_92']=New_dados['Media_9'].shift(1)
        #New_dados['Media_212']=New_dados['Media_21'].shift(1)
        New_dados.dropna(inplace=True)
        New_dados2 = New_dados[['upperband', 'RSI', 'macd', 'macdsignal', 'slowk', 
                         'slowd', 'macd2', 'macdsignal2', 'slowk2',
                         'slowd2']]
        #New_dados2.dropna(inplace=True)
    return New_dados2,New_dados

In [ ]:
def test(modo):
    if modo=='Compra':
        New_dados2,New_dados=backtest('Compra')
        New_dados2.shape
        for i in range(New_dados2.shape[0]):
            pred = rfc1.predict(New_dados2[i:i+1])
            if pred[0]!=0 and i < 50:
                print('Ordem de compra emitida!!!',New_dados['time'][i:i+1].item())
                input()
    if modo=='Venda':
        New_dados2,New_dados=backtest('Venda')
        New_dados2.shape
        for i in range(New_dados2.shape[0]):
            pred = rfc2.predict(New_dados2[i:i+1])
            if pred[0]!=0 and i < 50:
                print('Ordem de venda emitida!!!',New_dados['time'][i:i+1].item())
                input()


In [ ]:
predt_cp =[]
New_dados2_cp,New_dados_cp=backtest('Compra')
for i in range(New_dados_cp.shape[0]):
    predt_cp.append(rfc1.predict(New_dados2_cp[i:i+1])[0])
New_dados_cp['Pred']=predt_cp
New_dados_cp['Retornp_total']=New_dados_cp.apply(lambda x: x['Retorno'] if x['Retorno']> 0 and x['Pred']==1 else x['Retorno'] if x['Retorno']< 0 and x['Pred']==1 else 0,axis=1)
New_dados_cp['Acum_compra']=New_dados_cp['Retornp_total'].cumsum()
print('Compra R$',round(float(New_dados_cp['Acum_compra'][-1:]),2))

In [ ]:
predt_vd =[]
New_dados2_vd,New_dados_vd=backtest('Venda')
for i in range(New_dados_vd.shape[0]):
    predt_vd.append(rfc2.predict(New_dados2_vd[i:i+1])[0])
New_dados_vd['Pred']=predt_vd
New_dados_vd['Retornp_total']=New_dados_vd.apply(lambda x: x['Retorno'] if x['Retorno']< 0 and x['Pred']==1 else x['Retorno'] if x['Retorno']> 0 and x['Pred']==1 else 0,axis=1)
New_dados_vd['Acum_compra']=New_dados_vd['Retornp_total'].cumsum()
print('Venda R$',round(float(New_dados_vd['Acum_compra'][-1:]),2))

In [ ]:
sns.histplot(New_dados_vd['Retornp_total'],kde=True);

In [ ]:
sns.histplot(New_dados_cp['Retornp_total'],kde=True);

In [ ]:
fig=px.line(New_dados_cp,x='time',y='Acum_compra')
fig.show()

In [ ]:
fig=px.line(New_dados_vd,x='time',y='Acum_compra')
fig.show()

In [ ]:
New_dados2_cp,New_dados_cp=backtest('Compra')
New_dados2_vd,New_dados_vd=backtest('Venda')

if rfc1.predict(New_dados2_cp[-1:])[0]==1:
    print("Emitir ordem de compra!! às:",New_dados_cp['time'][-1:].item())
if rfc2.predict(New_dados2_vd[-1:])[0]==1:
    print("Emitir ordem de venda!!às:",New_dados_vd['time'][-1:].item())

Emitir ordem de venda!!às: 2021-06-28 17:45:00
